In [13]:
import os,sys,json,re
sys.path.insert(0, '../../scripts/')
from utils.io import tofile, fromfile, prettyjson
import ase
from ase.io import read
from ase.visualize import view
import numpy as np
import ubjson, json

In [2]:
!ls repo-fit-bulk/

dispts_quip_params.xml			mebox-minimal-oos-T188-pbe0-mbdint.xyz
fit_mbd_soap.sh				mebox-minimal-oos-T188-pbe-b1b.xyz
fit_soap_box.sh				mebox-minimal-oos-T188-pbe-mbdint.xyz
mebox-minimal-oos-T110-pbe0-b1b.xyz	mebox-minimal-pbe0-b1b.xyz
mebox-minimal-oos-T110-pbe0-mbdint.xyz	mebox-minimal-pbe0-pbeotf-mbdint.xyz
mebox-minimal-oos-T110-pbe-b1b.xyz	mebox-minimal-pbe-b1b.xyz
mebox-minimal-oos-T110-pbe-mbdint.xyz	mebox-minimal-pbe-mbdint.xyz
mebox-minimal-oos-T188-pbe0-b1b.xyz	README


In [2]:
fn = '/Users/iMac/Downloads/input.data'
with open(fn, 'r') as f:
    data = f.read().encode('unicode-escape').decode()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/iMac/Downloads/input.data'

In [111]:
patterns = {
    'struture':r'(?<=begin).*?(?=end)',
    'lattice':r'(?<=lattice).*?(?=\\n)',
    'atom':r'(?<=atom).*?(?=\\n)',
   'float':r"[-+]?[0-9]*\.?[0-9]+",
    'energy':r"energy\s+([-+]?[0-9]*\.?[0-9]+)\s+\\n",
   'species':r"[BCFHIKNOPSUVWY]|A[cglmrstu]|B[aehikr]|C[adeflmnorsu]|D[bsy]|E[rsu]|F[elmr]|G[ade]|H[efgos]|I[nr]|Kr|L[airuv]|M[dgnot]|N[abdeiop]|Os|P[abdmortu]|R[abefghnu]|S[bcegimnr]|T[abcehilm]|Uu[opst]|Xe|Yb|Z[nr]",
           }
frames = []
for res in results:
    ddd = re.findall(patterns['lattice'] , res)
    lat = []
    for d in ddd:
        lat.append(list(map(float,re.findall(patterns['float'] , d))))
    
    ddd = re.findall(patterns['atom'] , res)
    pos,sym,force = [],[],[]
    for d in ddd:
        aa = d.split()
        pos.append(list(map(float,aa[:3])))
        force.append(list(map(float,aa[-3:])))
        aa = aa[3]
        sym.append(aa)
    energy = float(re.findall(patterns['energy'] , res)[0])
    frame = ase.Atoms(positions=pos,symbols=sym,cell=lat,pbc=True)
    frame.info['energy'] = energy
    frame.set_array('forces',np.array(force))
    frame.wrap(eps=1e-11)
    frames.append(frame)

In [9]:
fn_out = '../methane_sulfonic.json'
tofile(fn_out,frames)

NameError: name 'frames' is not defined

In [168]:
frames = fromfile(fn_out)

In [169]:
len(frames)

4141

In [14]:
'Asdf'.lower()

'asdf'

In [21]:
fns = {'molecular_crystals':'CSD1000-r.xyz',
       'silicon_bulk':'gp_iter6_sparse9k.xml.xyz',
       'qm9':'qm9_5000.xyz',
       'methane_liquid':'repo-fit-bulk/mebox-minimal-pbe0-b1b.xyz'
}
fns = {'silicon_bulk':'gp_iter6_sparse9k.xml.xyz',}
for name,fn in fns.items():
    frames = read(fn,':')
    for frame in frames:
        if name == 'qm9':
            frame.set_pbc(False)
            frame.set_cell(np.eye(3)*12)
            frame.center(about=[6,6,6])
        frame.wrap(eps=1e-11)
        aa = {}
        for k,v in frame.info.items():
            aa[k.lower()] = v
        frame.info = aa
        aa = {}
        for k,v in frame.arrays.items():
            aa[k.lower()] = v
        frame.arrays = aa
    print(name,len(frames))
    fn_out = '../'+name+'.json'
    tofile(fn_out,frames)

silicon_bulk 2475


In [12]:
fn = '../'+'qm9'+'.json'
frames = read(fn,':')
y = np.load('qm9_eV_5000.dHf_peratom.npy')
for ii,ff in enumerate(frames):
    ff.info['dHf_peratom'] = y[ii]
    # pp = ff.get_positions()
    # if np.any(pp > 12) or np.any(pp < 0):
    #     print(ii,'%%%%%%%%%%%%')
    #     print(pp)
tofile(fn, frames)

array([-4.17287931, -4.55281707, -4.37783951, ..., -4.22765394,
       -4.2937652 , -4.36285139])

In [20]:
from ase.neighborlist import neighbor_list
path = '../'
fns = ['qm9.json', 'molecular_crystals.json', 'silicon_bulk.json', 
       'methane_liquid.json', 'methane_sulfonic.ubjson']
sp = {
    # 'qm9.json' : {'st':0,'nd':500, 'c':4, 'o':'qm9_0_500.json'}, 
    'molecular_crystals.json' : {'lg':100, 'c':4, 'o':'molecular_crystals_100.json'}, 
    # 'silicon_bulk.json' : {'st':600,'nd':700, 'c':4, 'o':'silicon_bulk_600_700.json'}, 
    # 'methane_liquid.json' : {'st':100,'nd':150, 'c':4, 'o':'methane_liquid_100_150.json'}, 
    #'methane_sulfonic.ubjson' : {'lg':150, 'c':4, 'o':'methane_sulfonic_150.json'}
}


for fn in sp:
    frames = fromfile(path+fn)
    s = sp[fn]
    ffs = frames
    nat = 0
    nneigh = 0
    sps = []
    sel = []
    ii = 0
    for ff in ffs:
        if ii >= s['lg']:
            break
        if np.product(ff.get_pbc()):
            cl = ff.get_cell_lengths_and_angles()[:3]
            if np.any(cl < 2 * s['c']):
                continue
            else:
                sel.append(ff)
                ii += 1
        i,j = ase.neighborlist.neighbor_list('ij', ff, s['c']) 
        nat += len(np.unique(i))
        nneigh += len(j)
        sps.extend(ff.get_atomic_numbers())
    print(fn, nat, nneigh, np.unique(sps))
    tofile(path+s['o'], sel)

molecular_crystals.json 14535 394656 [1 6 7 8]


In [6]:
view(sel)

In [33]:
from rascal.neighbourlist.base import _neighbourlist_list
_neighbourlist_list

['centers',
 'neighbourlist',
 'centercontribution',
 'strict',
 'maxorder',
 'halflist',
 'fulllist']

In [6]:
frames = fromfile('../molecular_crystals_50.json')
view(frames)

In [4]:
len(frames)

1000

In [31]:
!ls ../

methane_liquid.json	 qm9_0_500.json  README.md
methane_sulfonic.ubjson  qm9.json	 silicon_bulk_600_700.json
molecular_crystals.json  raw_data	 silicon_bulk.json
